In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, auc, roc_curve, precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
import os
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# !pip install transformers datasets accelerate evaluate scikit-learn matplotlib seaborn
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Getting distillation dataset

In [ ]:
class CyberbullyingDataset(Dataset):
  def __init__(self, texts: List[str], hard_labels: List[int], soft_labels: List[List[float]], tokenizer, max_length: int = 128):
    self.texts = texts
    self.hard_labels = hard_labels
    self.soft_labels = soft_labels
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    encoding = self.tokenizer(text,truncation=True,padding='max_length',max_length=self.max_length,return_tensors='pt')

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(self.hard_labels[idx], dtype=torch.long),
        'soft_labels': torch.tensor(self.soft_labels[idx], dtype=torch.float)
        }

## Trainer

In [ ]:
class DistillationTrainer(Trainer):
  def __init__(self, temperature: float = 2.0, alpha: float = 0.3, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.temperature = temperature
    self.alpha = alpha

  def compute_loss(self, model, inputs, return_outputs=False, **kwargs):

    labels = inputs.get("labels")
    soft_labels = inputs.get("soft_labels")

    # create a new dict withouot labels and soft_labels and pass it in the model
    outputs = model(**{k: v for k, v in inputs.items() if k not in ["labels", "soft_labels"]})
    logits = outputs.get("logits")

    ce_loss = F.cross_entropy(logits, labels)

    # Soft loss (KL divergence with teacher probabilities)
    student_probs = F.log_softmax(logits / self.temperature, dim=-1)
    teacher_probs = soft_labels

    teacher_probs = teacher_probs / teacher_probs.sum(dim=-1, keepdim=True)

    kl_loss = F.kl_div(student_probs, teacher_probs, reduction='batchmean')
    soft_loss = kl_loss * (self.temperature ** 2)


    total_loss = self.alpha * ce_loss + (1 - self.alpha) * soft_loss

    return (total_loss, outputs) if return_outputs else total_loss


In [ ]:
def load_distillation_data(json_path: str) -> Tuple[List[str], List[int], List[List[float]]]:
  """Load the teacher-labeled dataset"""
  with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

  texts = [item['text'] for item in data]
  hard_labels = [item['label_hard'] for item in data]
  soft_labels = [item['p_teacher'] for item in data]

  print(f"Loaded {len(texts)} samples")
  print(f"Label distribution: {pd.Series(hard_labels).value_counts().to_dict()}")

  return texts, hard_labels, soft_labels

In [ ]:
def compute_metrics(eval_pred):
  """Compute evaluation metrics"""
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  # Basic metrics
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

  # ROC AUC (using probabilities)
  # probs = F.softmax(torch.tensor(eval_pred.predictions), dim=-1).numpy()
  # probs = torch.softmax(torch.tensor(predictions), dim=-1).numpy()
  # pos_probs = probs[:, 1]
  probs = F.softmax(torch.tensor(eval_pred.predictions), dim=-1).numpy()
  pos_probs = probs[:, 1]
  try:
    # roc_auc = roc_auc_score(labels, probs[:, 1])
    # For ROC AUC
    fpr, tpr, _ = roc_curve(labels, pos_probs)
    roc_auc = auc(fpr, tpr)

    # For PR AUC
    # precision_curve, recall_curve, _ = precision_recall_curve(labels, pos_probs)
    # pr_auc = auc(recall_curve, precision_curve)

  except ValueError:
    roc_auc = 0.5

  return {
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1': f1,
      'roc_auc': roc_auc
      }

In [ ]:
def train_student_model(
    json_path: str,
    model_name: str = "nlpaueb/bert-base-greek-uncased-v1",
    output_dir: str = "./cyberbullying_bert",
    max_length: int = 128,
    batch_size: int = 16,
    num_epochs: int = 7,
    learning_rate: float = 2e-5,
    temperature: float = 2.0,
    alpha: float = 0.5,
    test_size: float = 0.2,
    validation_size: float = 0.1
    ):

  print("Starting BERT Training with Knowledge Distillation")
  print(f"Model: {model_name}")

  texts, hard_labels, soft_labels = load_distillation_data(json_path)

  X_temp, X_test, y_temp, y_test, soft_temp, soft_test = train_test_split(
      texts, hard_labels, soft_labels, test_size=test_size, random_state=42, stratify=hard_labels
      )

  X_train, X_val, y_train, y_val, soft_train, soft_val = train_test_split(
      X_temp, y_temp, soft_temp, test_size=validation_size/(1-test_size), random_state=42, stratify=y_temp
      )

  print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

  tokenizer = AutoTokenizer.from_pretrained(model_name)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding=True,return_tensors="pt")

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name,
      num_labels=2,
      problem_type="single_label_classification"
      )

  train_dataset = CyberbullyingDataset(X_train, y_train, soft_train, tokenizer, max_length)
  val_dataset = CyberbullyingDataset(X_val, y_val, soft_val, tokenizer, max_length)
  test_dataset = CyberbullyingDataset(X_test, y_test, soft_test, tokenizer, max_length)

  training_args = TrainingArguments(
      output_dir=output_dir,
      num_train_epochs=num_epochs,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      warmup_steps=100,
      weight_decay=0.01,
      logging_dir=f'{output_dir}/logs',
      logging_strategy="steps",
      eval_strategy ="steps",
      save_strategy="steps",
      load_best_model_at_end=True,
      metric_for_best_model="f1",
      greater_is_better=True,
      save_total_limit=3,
      report_to=None,  # Disable wandb
      learning_rate=learning_rate,
      fp16=True,
      seed=33,
      remove_unused_columns=False
      )

  trainer = DistillationTrainer(
        temperature=temperature,
        alpha=alpha,
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

  print("Starting fine tuning...")
  trainer.train()

  # Evaluate on test set
  print("Evaluating on test set...")
  test_results = trainer.evaluate(test_dataset)

  print("\n=== FINAL TEST RESULTS ===")
  for key, value in test_results.items():
    if key.startswith('eval_'):
      metric_name = key.replace('eval_', '')
      print(f"{metric_name}: {value:.4f}")

  print(f"Saving model to {output_dir}")
  trainer.save_model()
  tokenizer.save_pretrained(output_dir)

  # Save training info
  training_info = {
      'model_name': model_name,
      'temperature': temperature,
      'alpha': alpha,
      'max_length': max_length,
      'test_results': test_results,
      'training_args': training_args.to_dict()
      }

  try:
      from google.colab import drive
      import os

      if not os.path.exists('/content/drive'):
        print("Mounting Google Drive...")
        drive.mount('/content/drive')
      else:
        print("Google Drive already mounted")

      drive_dir = '/content/drive/MyDrive/cyshield'
      os.makedirs(drive_dir, exist_ok=True)

      print(f"Saving model & tokenizer to {drive_dir}")
      trainer.save_model(drive_dir)
      tokenizer.save_pretrained(drive_dir)

  except ImportError:
    print("Saving locally")



  with open(f"{output_dir}/training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

  print("Training complete!")
  return trainer, test_results

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
json_path = '/content/drive/MyDrive/cyshield/combined_teacher_labels.json'

trainer, results = train_student_model(
    json_path=json_path,
    model_name="nlpaueb/bert-base-greek-uncased-v1",
    output_dir="./cyberbullying_bert",
    max_length=128,
    batch_size=8,
    num_epochs=9,
    learning_rate=1e-5,
    temperature=1.5,
    alpha=0.9,       # Balance: 30% hard loss, 70% soft loss
    )

print("\nTraining finished!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting BERT Training with Knowledge Distillation
Model: nlpaueb/bert-base-greek-uncased-v1
Loaded 2568 samples
Label distribution: {0: 1787, 1: 781}
Train: 1797, Val: 257, Test: 514


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting fine tuning...


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
500,0.624200,0.590833,0.852140,0.738095,0.794872,0.765432,0.921179
1000,0.497200,0.643040,0.879377,0.747368,0.910256,0.820809,0.892136
1500,0.451900,0.614788,0.879377,0.747368,0.910256,0.820809,0.901518
2000,0.420500,0.585617,0.898833,0.802326,0.884615,0.841463,0.903058


Evaluating on test set...



=== FINAL TEST RESULTS ===
loss: 0.5912
accuracy: 0.8911
precision: 0.8247
recall: 0.8141
f1: 0.8194
roc_auc: 0.8969
runtime: 1.1015
samples_per_second: 466.6510
steps_per_second: 59.0120
Saving model to ./cyberbullying_bert
Google Drive already mounted
Saving model & tokenizer to /content/drive/MyDrive/cyshield
Training complete!

Training finished!
